In [1]:
import numpy as np
import os
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, auc, roc_curve

from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping

from Libs.models import make_lstm
from Libs.config import inter_extra_data_folder, models_InterExtra_folder
from Libs.load_data import DataLoader, get_dataset_split
from Libs.keras_f1score import f1_m

2023-03-28 07:18:24.695558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 07:18:24.835191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 07:18:24.835220: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-28 07:18:25.738076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

A biggest assumption when training ANNs is the following: 

"We assume that training sets and test sets contains independent and identically distributed samples from the same unknown distribution $p_{data}(x,y)$"

This is a very important assumption that in general affect the performance ANNs, in particular classifier ones. We could, indeed, explore what can happen if we violete the following assumption. This a relevant application case, for exaple in cases when the generation parameters are not known.

In [2]:
# flag which says if we need to re-train the models for this experiment, default is False
override_lstm = False
# flag which says if the time series need to be standardize or not, default is False 
F_std = False

# initialize data loader
data_loader = DataLoader(run=100, N=1000, s=0.5, t=[0.01, 0.1, 0.5, 3], d=0.2, m=1, 
                         override=False, folder=inter_extra_data_folder)
# get the grid
grid_X, grid_y = data_loader.get_grid()
# get params dictionary
params = data_loader.get_params()

grid_X.shape, grid_y.shape

((100, 1, 4, 1, 1, 1000), (100, 1, 4, 1, 1, 1000))

The following function is used to split the dataset in training, validation and test set according to a given parameter modality (i.e. `mide`). If this parameter is not `all` the training and validation data and test data will be generater in two different ways (i.e. for inteporlation extreme values of theta will be used for training and internal parameters for test, while for extrapolation extreme values of theta will be used for test and internal parameters for training)

In [3]:
def get_data_set(data1, mode, data2=None, F_std=False, overlap_size=15):
    assert mode in ['all', 'interpolation', 'extrapolation']    

    # params commons
    dataset_split_params = {
        'window_size': 20, # how large is the window
        'overlap_size': overlap_size, # how many time interval of overlap there is between the windows
        'label_treshold': 1, # how many labels have to be at 1 in the window_size to consider the current window as a flare
        'split_on_run': True, # if True the windows of a run cannot be on different sets
        'shuffle_run': False, # if True shuffles the order of the runs before computing the windows
        'shuffle_window': False, # if True shuffles the order of the windows in the resulting dataframes
        'test_size': 0.3, # size of the test set expressed in percentage
        'val_size': 0.2, # size of the validation set expressed in percentage, considered only if get_validation is True
        'get_validation': True, # if True the output would be train,val,test set, otherwise it would be train,test
        'random_state': 42, # sets the seed for reproducibility
        'get_info':False, # Extends windows dataframe with infos on the params, window_range and label_range
        'params': params # needed when get_info is True
    }

    if mode in ['interpolation', 'extrapolation']:
        assert not data2 is None
        grid_X_train, grid_y_train = data1
        grid_X_test, grid_y_test   = data2
        # get the train and validation set, selecting the index for grid given the interpolation assuption
        # notice that theta is the third parameter
        df_train, df_val, _ = get_dataset_split(grid_X_train, grid_y_train, **dataset_split_params)
        # get the test set, selecting the index for grid given the interpolation assuption
        # notice that theta is the third parameter
        _, _, df_test = get_dataset_split(grid_X_test, grid_y_test, **dataset_split_params)
    elif mode in ['all']:
        grid_X, grid_y = data1
        # get all the dataset from a single list
        df_train, df_val, df_test = get_dataset_split(grid_X, grid_y, **dataset_split_params)
    else:
        raise NotImplemented()
    
    # number of classes
    print('Training set:')
    train_counts = df_train['future_flare'].value_counts()
    print(train_counts, '\n')
    print('validation set:')
    val_counts = df_val['future_flare'].value_counts()
    print(val_counts, '\n')
    print('Test set:')
    test_counts = df_test['future_flare'].value_counts()
    print(test_counts, '\n')
    print('Total:')
    total_counts = train_counts.add(val_counts).add(test_counts)
    print(total_counts, '\n')
    print()
    
    # compute the initial bias to pass then to the model
    initial_bias = [np.log(train_counts[0]/train_counts[1])]

    # check the shape
    X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
    X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
    X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
    print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
    print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

    # finally, if requested, standardize the dataset
    if F_std:
        # Standardize Data
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_std = scaler.transform(X_train)
        X_val_std = scaler.transform(X_val)
        X_test_std = scaler.transform(X_test)
    else:
        X_train_std = X_train
        X_val_std = X_val
        X_test_std = X_test


    # finally return the dataset
    return X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, initial_bias

The following function will evaluate the score obtained with predictions on validation set and test set

In [4]:
def eval(model, X_val, y_val, X_test, y_test):
    # Validation set
    y_pred = np.round(model.predict(X_val), 0)
    print("### Evaluation on validation set ###")
    print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
    print("F1 score: %.2f" % (f1_score(y_pred, y_val, average='macro')))
    fpr, tpr, _ = roc_curve(y_val, y_pred, pos_label=1)
    print('AUC:', auc(fpr, tpr))
    #Create confusion matrix and normalizes it over predicted (columns)
    cm = confusion_matrix(y_val, y_pred)
    print(cm)

    print()
    
    # Test set
    y_pred = np.round(model.predict(X_test), 0)
    print("### Evaluation on test set ###")
    print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
    print("F1 score: %.2f" % (f1_score(y_pred, y_test, average='macro')))
    fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=1)
    print('AUC:', auc(fpr, tpr))
    #Create confusion matrix and normalizes it over predicted (columns)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

# LSTM model with multiple all theta parameters

Let's start considering for both training and test set the same data distribution, namely dataset produced with all the four theta parameter

In [5]:
X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set((grid_X.copy(), grid_y.copy()), 'all', F_std=F_std)

Training set:
0    25172
1    12656
Name: future_flare, dtype: int64 

validation set:
0    10367
1     5845
Name: future_flare, dtype: int64 

Test set:
0    15145
1     8015
Name: future_flare, dtype: int64 

Total:
0    50684
1    26516
Name: future_flare, dtype: int64 


X ## Train: (37828, 20) Val: (16212, 20) Test: (23160, 20)
y ## Train: (37828,) Val: (16212,) Test: (23160,)


In [6]:
lstm_all_path = os.path.join(models_InterExtra_folder, 'std', "LSTM_allTheta_checkpoint.h5") if F_std else \
                os.path.join(models_InterExtra_folder, 'not_std', "LSTM_allTheta_checkpoint.h5")
model_all = make_lstm((X_train.shape[1], 1), output_bias=initial_bias)
model_all.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=[f1_m, 'accuracy'])
print(model_all.summary())

2023-03-28 07:18:27.599354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-28 07:18:27.599452: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-28 07:18:27.599481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (44910f15382a): /proc/driver/nvidia/version does not exist
2023-03-28 07:18:27.599801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               3520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 30)                1230      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
if override_lstm:
    epochs = 20
    batch_size = 32
    # define callbacks
    callbacks = [
        ModelCheckpoint(
            lstm_all_path, save_weights_only=True, monitor="val_loss"
        ),
        EarlyStopping(monitor="val_loss", patience=3, verbose=1),
    ]
    # fit model
    model_all.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, y_val),
        verbose=1,
    )
model_all.load_weights(lstm_all_path)

Validation set results

In [8]:
eval(model_all, X_val, y_val, X_test, y_test)

507/507 [==============================] - 3s 6ms/step
### Evaluation on validation set ###
Accuracy: 0.75
F1 score: 0.73
AUC: 0.7260208434293755
[[8262 2105]
 [2016 3829]]

724/724 [==============================] - 4s 5ms/step
### Evaluation on test set ###
Accuracy: 0.74
F1 score: 0.72
AUC: 0.7157500988057429
[[12098  3047]
 [ 2944  5071]]


# Interpolation

Same model as before, but now we are fitting only in the dataset coming from the generation with using just the extreme parameters: 

$\theta=0.01$ and $\theta=3$

and a fraction of the other dataset, coming from $\theta=0.1$ and $\theta=0.5$ as test set

In [9]:
p = 'theta'
# train configurations
theta_train_list     = [0.01, 3]
theta_train_list_idx = [params[p].index(t) for t in theta_train_list]
data_train = (grid_X[:,:,theta_train_list_idx,:,:,:].copy(), grid_y[:,:,theta_train_list_idx,:,:,:].copy())
# test configuration
theta_test_list      = [0.1, 0.5]
theta_test_list_idx  = [params[p].index(t) for t in theta_test_list]
data_test = (grid_X[:,:,theta_test_list_idx,:,:,:].copy(), grid_y[:,:,theta_test_list_idx,:,:,:].copy())

X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set(data_train, 'interpolation', data2=data_test, F_std=F_std)

Training set:
0    13576
1     5338
Name: future_flare, dtype: int64 

validation set:
0    5630
1    2476
Name: future_flare, dtype: int64 

Test set:
0    7015
1    4565
Name: future_flare, dtype: int64 

Total:
0    26221
1    12379
Name: future_flare, dtype: int64 


X ## Train: (18914, 20) Val: (8106, 20) Test: (11580, 20)
y ## Train: (18914,) Val: (8106,) Test: (11580,)


In [10]:
lstm_interp_path = os.path.join(models_InterExtra_folder, "std", "LSTM_intrpTheta_checkpoint.h5") if F_std else \
                   os.path.join(models_InterExtra_folder, "not_std", "LSTM_intrpTheta_checkpoint.h5")
model_interp = make_lstm((X_train.shape[1], 1), output_bias=initial_bias)
model_interp.compile(loss='binary_crossentropy', 
                     optimizer='adam', 
                     metrics=[f1_m, 'accuracy'])
print(model_interp.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 30)                1230      
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
if override_lstm:
    epochs = 20
    batch_size = 32
    # define callbacks
    callbacks = [
        ModelCheckpoint(
            lstm_interp_path, save_weights_only=True, monitor="val_loss"
        ),
        EarlyStopping(monitor="val_loss", patience=3, verbose=1),
    ]
    # fit model
    model_interp.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, y_val),
        verbose=1,
    )
model_interp.load_weights(lstm_interp_path)

In [12]:
eval(model_interp, X_val, y_val, X_test, y_test)

254/254 [==============================] - 1s 5ms/step
### Evaluation on validation set ###
Accuracy: 0.74
F1 score: 0.72
AUC: 0.7451789398474018
[[4100 1530]
 [ 589 1887]]

362/362 [==============================] - 2s 5ms/step
### Evaluation on test set ###
Accuracy: 0.75
F1 score: 0.73
AUC: 0.7195498770823592
[[6000 1015]
 [1900 2665]]


# Extrapolation

Same model as before, but now we are fitting only in the dataset coming from the generation without using the extreme parameters: 

$\theta=0.1$ and $\theta=0.5$

In [13]:
p = 'theta'
# train configurations
theta_train_list     = [0.1, 0.5]
theta_train_list_idx = [params[p].index(t) for t in theta_train_list]
data_train = (grid_X[:,:,theta_train_list_idx,:,:,:].copy(), grid_y[:,:,theta_train_list_idx,:,:,:].copy())
# test configuration
theta_test_list      = [0.01, 3]
theta_test_list_idx  = [params[p].index(t) for t in theta_test_list]
data_test = (grid_X[:,:,theta_test_list_idx,:,:,:].copy(), grid_y[:,:,theta_test_list_idx,:,:,:].copy())

X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set(data_train, 'extrapolation', F_std=F_std,
                                                                            data2=data_test)

Training set:
0    11596
1     7318
Name: future_flare, dtype: int64 

validation set:
0    4737
1    3369
Name: future_flare, dtype: int64 

Test set:
0    8130
1    3450
Name: future_flare, dtype: int64 

Total:
0    24463
1    14137
Name: future_flare, dtype: int64 


X ## Train: (18914, 20) Val: (8106, 20) Test: (11580, 20)
y ## Train: (18914,) Val: (8106,) Test: (11580,)


In [14]:
lstm_extrp_path = os.path.join(models_InterExtra_folder, "std", "LSTM_extrpTheta_checkpoint.h5") if F_std else \
                   os.path.join(models_InterExtra_folder, "not_std", "LSTM_extrpTheta_checkpoint.h5")
model_extrp = make_lstm((X_train.shape[1], 1), output_bias=initial_bias)
model_extrp.compile(loss='binary_crossentropy', 
                    optimizer='adam', 
                    metrics=[f1_m, 'accuracy'])
print(model_extrp.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 30)                1230      
                                                                 
 dense_7 (Dense)             (None, 10)                310       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
if override_lstm:
    epochs = 20
    batch_size = 32
    # define callbacks
    callbacks = [
        ModelCheckpoint(
            lstm_extrp_path, save_weights_only=True, monitor="val_loss"
        ),
        EarlyStopping(monitor="val_loss", patience=3, verbose=1),
    ]
    # fit model
    model_extrp.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, y_val),
        verbose=1,
    )
model_extrp.load_weights(lstm_extrp_path)

In [16]:
eval(model_extrp, X_val, y_val, X_test, y_test)

254/254 [==============================] - 2s 5ms/step
### Evaluation on validation set ###
Accuracy: 0.75
F1 score: 0.73
AUC: 0.7277705811903826
[[4132  605]
 [1404 1965]]

362/362 [==============================] - 2s 5ms/step
### Evaluation on test set ###
Accuracy: 0.74
F1 score: 0.67
AUC: 0.6649606930851917
[[6924 1206]
 [1800 1650]]


## Further investigation in extrapolation with greater overlap

In [17]:
X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set(data_train, 'extrapolation', F_std=F_std, 
                                                                            data2=data_test, overlap_size=19)

Training set:
0    57784
1    36394
Name: future_flare, dtype: int64 

validation set:
0    23613
1    16749
Name: future_flare, dtype: int64 

Test set:
0    40453
1    17207
Name: future_flare, dtype: int64 

Total:
0    121850
1     70350
Name: future_flare, dtype: int64 


X ## Train: (94178, 20) Val: (40362, 20) Test: (57660, 20)
y ## Train: (94178,) Val: (40362,) Test: (57660,)


In [18]:
lstm_extrp19_path = os.path.join(models_InterExtra_folder, "std", "LSTM_extrpTheta19_checkpoint.h5") if F_std else \
                    os.path.join(models_InterExtra_folder, "not_std", "LSTM_extrpTheta19_checkpoint.h5")
model_extrp19 = make_lstm((X_train.shape[1], 1), output_bias=initial_bias)
model_extrp19.compile(loss='binary_crossentropy', 
                      optimizer='adam', 
                      metrics=[f1_m, 'accuracy'])
print(model_extrp19.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 30)                1230      
                                                                 
 dense_10 (Dense)            (None, 10)                310       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
if override_lstm:
    epochs = 20
    batch_size = 32
    # define callbacks
    callbacks = [
        ModelCheckpoint(
            lstm_extrp19_path, save_weights_only=True, monitor="val_loss"
        ),
        EarlyStopping(monitor="val_loss", patience=3, verbose=1),
    ]
    # fit model
    model_extrp19.fit(
        X_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(X_val, y_val),
        verbose=1,
    )
model_extrp19.load_weights(lstm_extrp19_path)

In [20]:
eval(model_extrp19, X_val, y_val, X_test, y_test)

1262/1262 [==============================] - 6s 5ms/step
### Evaluation on validation set ###
Accuracy: 0.75
F1 score: 0.74
AUC: 0.7305525378749167
[[20472  3141]
 [ 6798  9951]]

1802/1802 [==============================] - 12s 7ms/step
### Evaluation on test set ###
Accuracy: 0.74
F1 score: 0.69
AUC: 0.6815823734257996
[[33748  6705]
 [ 8106  9101]]


# Bibliography
\[1\] _On the distribution of fluxes of gamma-ray blazars: hints for a stochastic process?_, Tavecchio et al., [https://arxiv.org/pdf/2004.09149.pdf](https://arxiv.org/pdf/2004.09149.pdf)
<!-- cite with: [\[1\]](https://arxiv.org/pdf/2004.09149.pdf)  -->
\[2\] _Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline_, Wang et al., [https://arxiv.org/abs/1611.06455](https://arxiv.org/abs/1611.06455)
<!-- cite with: [\[2\]](https://arxiv.org/abs/1611.06455)  -->
\[3\] _Solar Flare Prediction Based on the Fusion of Multiple Deep-learning Models_, Tang et al., [https://iopscience.iop.org/article/10.3847/1538-4365/ac249e/meta](https://iopscience.iop.org/article/10.3847/1538-4365/ac249e/meta)
<!-- cite with: [\[3\]](https://iopscience.iop.org/article/10.3847/1538-4365/ac249e/meta)  -->
\[4\] _Predicting Solar Energetic Particles Using SDO/HMI Vector Magnetic Data Products and a Bidirectional LSTM Network_, Abduallah et al., [https://iopscience.iop.org/article/10.3847/1538-4365/ac5f56/meta](https://iopscience.iop.org/article/10.3847/1538-4365/ac5f56/meta)
<!-- cite with: [\[4\]](https://iopscience.iop.org/article/10.3847/1538-4365/ac5f56/meta) -->